# Definicije tipov

In [1]:
(* Ko se v Jupytru prvič požene OCaml, program Findlib izpiše neko sporočilo.
   Da se to sporočilo ne bi videlo v zapiskih, je tu ta celica, ki sproži izpis,
   vendar ima nastavljeno, da je v zapiskih v celoti skrita. *)

Poleg bogatega nabora vgrajenih tipov si tipe v OCamlu lahko definiramo tudi sami.

## Okrajšave tipov

Najenostavnejši način za definicijo tipov so okrajšave obstoječih tipov. Na primer, tip za $\mathbb{R}^3$ si lahko definiramo kot:

In [2]:
type r3 = float * float * float

type r3 = float * float * float


Tako kot na primer vgrajeni tip `list` lahko tudi naši tipi vsebujejo parametre:

In [3]:
type 'a zaporedje = int -> 'a

type ('k, 'v) slovar = ('k * 'v) list

type 'a zaporedje = int -> 'a


type ('k, 'v) slovar = ('k * 'v) list


Če tip sprejme več parametrov (na primer slovar ima tako tip ključev kot tip vrednosti), jih lahko naštejemo v oklepajih.

## Zapisni tipi

Recimo, da si definiramo kompleksna števila s pari realnih:

In [4]:
type kompleksno = float * float

type kompleksno = float * float


Kako bi izračunali absolutno vrednost kompleksnega števila? Ena možnost je:

In [5]:
let abs (x, y) = sqrt (x ** 2. +. y ** 2.)

val abs : float * float -> float = <fun>


Toda če smo v mislih imeli polarni zapis, je pravilna definicija:

In [6]:
let abs (r, _fi) = r

val abs : 'a * 'b -> 'a = <fun>


Zmešnjavi se lahko izognemo, če obe komponenti poimenujemo. V OCamlu to storimo z zapisnimi tipi, ki jih podamo tako, da naštejemo imena polj ter njihove tipe:

In [7]:
type kartezicno = {re : float; im : float}
type polarno = {radij : float; kot : float}

type kartezicno = { re : float; im : float; }


type polarno = { radij : float; kot : float; }


Vrednosti tipov pišemo podobno, le da jih podamo z `=`:

In [8]:
let i = {re = 0.0; im = 1.0}

val i : kartezicno = {re = 0.; im = 1.}


Do komponent dostopamo z `zapis.polje`:

In [9]:
let abs z = sqrt (z.re ** 2. +. z.im ** 2.)

val abs : kartezicno -> float = <fun>


In [10]:
let abs' z = z.radij

val abs' : polarno -> float = <fun>


Z zapisom `{zapis with polje1 = vrednost1, ...}` ustvarimo nov zapis, ki ima z izjemo naštetih vrednosti polja enaka prvotnemu:

In [11]:
let konjugiraj z = {z with im = -. z.im}

val konjugiraj : kartezicno -> kartezicno = <fun>


In [12]:
konjugiraj i

- : kartezicno = {re = 0.; im = -1.}


Kljub temu, da zapise pišemo podobno kot Pythonove slovarje, gre za popolnoma različni strukturi. Zapisi so v resnici kartezični produkti, le da so komponente poimenovane, imena polj pa niso vrednosti, ki bi si jih lahko podajali naokoli.

## Vsote

Najzanimivejši tip, ki ga lahko definiramo, pa so _vsote_. Podamo jih tako, da naštejemo možne variante, od katerih je vsaka podana s svojim _konstruktorjem_. Če se želimo omejiti na fiksno množico velikosti oblačil, lahko na primer napišemo enostavno vsoto s petimi variantami:

In [13]:
type velikost = XS | S | M | L | XL

type velikost = XS | S | M | L | XL


Tedaj bo tip imel natanko pet možnih vrednosti in OCaml nas bo opozoril, če poskusimo uporabiti nenavedeno varianto:

In [14]:
[XS; XS; M; S; XL; L; L]

- : velikost list = [XS; XS; M; S; XL; L; L]


In [15]:
[XS; XS; L; M; M; XM]

error: compile_error

Vsak izmed naštetih konstruktorjev lahko sprejme tudi argumente vnaprej določenega tipa:

In [16]:
type geometrijski_objekt =
  | Tocka
  | Krog of float
  | Pravokotnik of float * float

type geometrijski_objekt =
    Tocka
  | Krog of float
  | Pravokotnik of float * float


In [17]:
[Tocka; Pravokotnik (1., 2.); Tocka; Krog 3.]

- : geometrijski_objekt list = [Tocka; Pravokotnik (1., 2.); Tocka; Krog 3.]


Tako kot vsote naštejemo po kosih, lahko prek `match` ali `function` po kosih tudi definiramo funkcije na njih.

In [18]:
let povrsina obj =
  match obj with
  | Tocka -> 0.
  | Krog r -> 3.14 *. r ** 2.
  | Pravokotnik (v, s) -> v *. s

val povrsina : geometrijski_objekt -> float = <fun>


In [19]:
let obseg =
  function
  | Tocka -> 0.
  | Krog r -> 2. *. 3.14 *. r
  | Pravokotnik (v, s) -> 2. *. (v +. s)

val obseg : geometrijski_objekt -> float = <fun>


## Tip `option`

Včasih imamo opravka s funkcijami, ki jih ne moremo povsod dobro definirati. Na primer, glava seznama je definirana samo za sestavljeni seznam. Če želimo, lahko naštejemo le to možnost, vendar bomo potem dobili napako ob izvajanju.

In [20]:
let slaba_glava (x :: _) = x

File "[20]", line 1, characters 16-28:


1 | let slaba_glava (x :: _) = x


                    ^^^^^^^^^^^^


Warning 8 [partial-match]: this pattern-matching is not exhaustive.


Here is an example of a case that is not matched:


[]


val slaba_glava : 'a list -> 'a = <fun>


In [21]:
slaba_glava [1; 2; 3]

- : int = 1


In [22]:
slaba_glava []

error: runtime_error

Varnejši način je, da uporabimo tip `'a option`, ki predstavlja morebitno vrednost tipa `'a`:

```ocaml
type 'a option = None | Some of 'a
```

Nato bi lahko glavo seznama definirali tako, da vedno vrne vrednost:

In [23]:
let glava = function
  | [] -> None
  | x :: _ -> Some x

val glava : 'a list -> 'a option = <fun>


In [24]:
glava [1; 2; 3]

- : int option = Some 1


In [25]:
glava []

- : 'a option = None


Če uporabimo tip `option`, nas tipi prisilijo, da obravnavamo robne primere. Na primer, prej bi lahko napisali:

In [26]:
let ali_je_slaba_glava_velika xs = slaba_glava xs > 100

val ali_je_slaba_glava_velika : int list -> bool = <fun>


Če uporabimo varnejši način, pa rezultata ne moremo neposredno primerjati s `100`:

In [27]:
let ali_je_glava_velika xs = glava xs > 100

error: compile_error

Tipi nas prisilijo, da obravnavamo vse primere in se odločimo, kaj naredimo, če podatka ni na voljo. Lahko bi se na primer odločili, da seznam brez glave nima velike glave:

In [28]:
let ali_je_glava_velika xs =
  match glava xs with
  | None -> false
  | Some x -> x > 100

val ali_je_glava_velika : int list -> bool = <fun>


Lahko bi se odločili tudi drugače, na primer da spet vrnemo morebitni odgovor, v vsakem primeru pa ne bomo pozabili na noben primer:

In [29]:
let ali_je_glava_velika xs =
  match glava xs with
  | None -> None
  | Some x -> Some (x > 100)

val ali_je_glava_velika : int list -> bool option = <fun>


Vsote so lahko definirane tudi rekurzivno. Takim tipom pravimo tudi _induktivni_ ali _algebrajski_ tipi. Najenostavnejši primer induktivnega tipa so naravna števila. Predstavimo jih z vsoto z dvema konstruktorjema `Nic` in `Naslednik`, pri čemer slednji sprejme en argument, ki je zopet naravno število.

In [30]:
type naravno = Nic | Naslednik of naravno

type naravno = Nic | Naslednik of naravno


Vsoto naravnih števil podamo z običajno rekurzivno definicijo:

In [31]:
let rec vsota m n =
  match m with
  | Nic -> n
  | Naslednik m' -> Naslednik (vsota m' n)

val vsota : naravno -> naravno -> naravno = <fun>


Še en že poznan primer induktivnega tipa so seznami. Vsak seznam je bodisi prazen, bodisi sestavljen iz glave in repa:

In [32]:
type 'a list =
  | Prazen
  | Sestavljen of 'a * 'a list

type 'a list = Prazen | Sestavljen of 'a * 'a list


Sedaj tudi vidimo, zakaj `::` lahko uporabljamo v vzorcih - ni namreč običajna funkcija za sestavljanje seznamov, temveč konstruktor tipa seznamov.

Induktivne tipe se pogosto uporablja za predstavitev izrazov določenega formalnega jezika. Na primer, aritmetične izraze gradimo iz števil ter aritmetičnih operacij. Take izraze bi lahko predstavili s tipom:

In [33]:
type izraz =
  | Stevilo of int
  | Plus of izraz * izraz
  | Minus of izraz
  | Krat of izraz * izraz

type izraz =
    Stevilo of int
  | Plus of izraz * izraz
  | Minus of izraz
  | Krat of izraz * izraz


Na primer, izrazu $-(5 \times (2 + 7))$ bi ustrezala vrednost

In [34]:
let i = Minus (
  Krat (Stevilo 5, Plus (Stevilo 2, Stevilo 7))
)

val i : izraz = Minus (Krat (Stevilo 5, Plus (Stevilo 2, Stevilo 7)))


Za vajo lahko napišete rekurzivno funkcijo `izracunaj : izraz -> int`, ki dani izraz prevori v njegovo vrednost. Na primer, za zgornji izraz bi funkcija vrnila `-45`.

Še en induktivni tip, ki ga bomo podrobneje spoznali v kratkem, pa so dvojiška drevesa. Dvojiško drevo je bodisi prazno bodisi ima koren, v katerem je shranjena vrednost, ter dva otroka, ki sta zopet drevesi, na primer (pri čemer praznih dreves ne kažemo):

![](slike/09-iskalna-drevesa/avl-drevo.png)

Tip dvojiških dreves podamo s tipom

In [35]:
type 'a drevo =
  | Prazno
  | Sestavljeno of 'a drevo * 'a * 'a drevo

type 'a drevo = Prazno | Sestavljeno of 'a drevo * 'a * 'a drevo
